In [21]:
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import random
import csv

In [ ]:
def reduce_resolution(data, interval=1):
    
    interval_values = {}

    for timestamp, value in data:
        # Find which interval this timestamp belongs to
        interval_index = int(timestamp // interval)

        # If not in add it
        if interval_index not in interval_values:
            interval_values[interval_index] = value
        else:
            # If any value is 1, set the interval to 1
            interval_values[interval_index] = max(interval_values[interval_index], value)

    # Convert to sorted list of (timestamp, value) pairs
    reduced_data = []
    for interval_index in sorted(interval_values):
        reduced_timestamp = interval_index * interval
        reduced_value = interval_values[interval_index]
        reduced_data.append((reduced_timestamp, reduced_value))

    return reduced_data

def compute_confusion_matrix(sensor_data, manual_data):
    print("\nComputing confusion matrix...")

    sensor_reduced = reduce_resolution(sensor_data, interval=1)
    manual_reduced = reduce_resolution(manual_data, interval=1)

    all_buckets = set(sensor_reduced.keys()) | set(manual_reduced.keys())

    TP = FP = TN = FN = 0

    for t in sorted(all_buckets):
        sensor_val = sensor_reduced.get(t, 0)
        manual_val = manual_reduced.get(t, 0)

        if sensor_val == 1 and manual_val == 1:
            TP += 1
        elif sensor_val == 1 and manual_val == 0:
            FP += 1
        elif sensor_val == 0 and manual_val == 0:
            TN += 1
        elif sensor_val == 0 and manual_val == 1:
            FN += 1

    confusion_matrix = {
        "TP": TP,
        "TN": TN,
        "FP": FP,
        "FN": FN,
    }
    return confusion_matrix


In [ ]:


# Simulate 10 seconds worth of data with 1000 samples
sampling_rate = 100  # 100 samples per second

num_samples = 1000  # Total 1000 samples
interval_between_samples = 10 / num_samples  # 10 seconds divided by 1000 samples

# Generate synthetic data: (timestamp in seconds, biased random 0 or 1)
data = []
last_one_timestamp = -float('inf')  # Initialize to negative infinity
group_size = random.randint(50, 150)  # Randomize the number of consecutive 1s in a group to simulate vibration intensity

for i in range(num_samples):
    timestamp = i * interval_between_samples
    # Ensure 1 occurs only if at least 2 seconds have passed since the last group of 1s
    if timestamp - last_one_timestamp >= 2 and random.random() < 0.2:  # 20% chance to start a group of 1s
        for _ in range(group_size):
            if len(data) < num_samples:  # Ensure we don't exceed the total number of samples
                data.append((timestamp, 1))
                timestamp += interval_between_samples
        last_one_timestamp = timestamp - interval_between_samples
    else:
        data.append((timestamp, 0))  # Bias towards 0

# Write to CSV
with open('test_data.csv', 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(['timestamp', 'value'])
    writer.writerows(data)

print("Sample data saved to 'test_data.csv'")


In [20]:
f = pd.read_csv('test_data.csv', sep=',')
#print(f.head())


# Reduce resolution to 1 microsecond intervals
reduced_data = reduce_resolution(data, interval=0.5)

print("Reduced data:")
for timestamp, value in reduced_data:
    print(f"Timestamp: {timestamp:.6f}, Value: {value}")

Reduced data:
Timestamp: 0.000000, Value: 1
Timestamp: 0.500000, Value: 1
Timestamp: 1.000000, Value: 0
Timestamp: 1.500000, Value: 0
Timestamp: 2.000000, Value: 0
Timestamp: 2.500000, Value: 0
Timestamp: 3.000000, Value: 1
Timestamp: 3.500000, Value: 1
Timestamp: 4.000000, Value: 0
Timestamp: 4.500000, Value: 0
Timestamp: 5.000000, Value: 0
Timestamp: 5.500000, Value: 0
Timestamp: 6.000000, Value: 1
Timestamp: 6.500000, Value: 1
Timestamp: 7.000000, Value: 0
Timestamp: 7.500000, Value: 0
Timestamp: 8.000000, Value: 0
Timestamp: 8.500000, Value: 0
Timestamp: 9.000000, Value: 0
Timestamp: 9.500000, Value: 0


In [ ]:
# plot confusion matrix

sns.set(style="whitegrid")
plt.figure(figsize=(10, 6))
plt.title("Sensor Data")
plt.xlabel("Time (s)")
plt.ylabel("Sensor Value")
